In [63]:
import pandas as pd
from ast import literal_eval
import numpy as np
from scipy.spatial.distance import euclidean, pdist, squareform
from os.path import join as os_join
import seaborn as sns
import matplotlib.pylab as plt
from pyvis import network as net


pd.set_option('display.max_colwidth', None)

In [64]:
similarity_path = "../data/processed/vis_dataset/similarity_combined.csv"
dataset_path = "../data/processed/vis_dataset/vis_data.csv"

output_path = "../reports/figures/ref_similarity/similarity_using_citeTo_and_citedBy.html"

In [65]:
dataset = pd.read_csv(dataset_path)

dataset["Title"] = dataset["Title"].astype(str)
dataset["AuthorNames"] = dataset["AuthorNames"].astype(str)


dataset.sample(2)

,Conference,Year,Title,DOI,id,cite_to_list,cited_by_list,Link,FirstPage,LastPage,...,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount_02-2020,XploreCitationCount - 2020-01,PubsCited,Award
579,InfoVis,2010,Evaluating the impact of task demands and block resolution on the effectiveness of pixel-based visualization,10.1109/TVCG.2010.150,579,[],"[1088, 1357]",http://dx.doi.org/10.1109/TVCG.2010.150,963,972,...,"Pixel-based visualization is a popular method of conveying large amounts of numerical data graphically. Application scenarios include business and finance, bioinformatics and remote sensing. In this work, we examined how the usability of such visual representations varied across different tasks and block resolutions. The main stimuli consisted of temporal pixel-based visualization with a white-red color map, simulating monthly temperature variation over a six-year period. In the first study, we included 5 separate tasks to exert different perceptual loads. We found that performance varied considerably as a function of task, ranging from 75% correct in low-load tasks to below 40% in high-load tasks. There was a small but consistent effect of resolution, with the uniform patch improving performance by around 6% relative to higher block resolution. In the second user study, we focused on a high-load task for evaluating month-to-month changes across different regions of the temperature range. We tested both CIE L*u*v* and RGB color spaces. We found that the nature of the change-evaluation errors related directly to the distance between the compared regions in the mapped color space. We were able to reduce such errors by using multiple color bands for the same data range. In a final study, we examined more fully the influence of block resolution on performance, and found block resolution had a limited impact on the effectiveness of pixel-based visualization.",Rita Borgo;Karl J. Proctor;Min Chen 0001;Heike Leitte;Tavi Murray;Ian M. Thornton,Rita Borgo;Karl Proctor;Min Chen;Heike Janicke;Tavi Murray;Ian Thornton,Swansea University;Swansea University;Swansea University;Heidelberg University;Swansea University;Swansea University,10.1109/VISUAL.1995.480803,"Pixel-based visualization, evaluation, user study, visual search, change detection",12.0,13.0,38.0,NaN
995,InfoVis,2014,Design Activity Framework for Visualization Design,10.1109/TVCG.2014.2346331,995,"[456, 874, 655, 755, 892]","[1125, 1146, 1176, 1359, 1428, 1469, 1524, 1606]",http://dx.doi.org/10.1109/TVCG.2014.2346331,2191,2200,...,"An important aspect in visualization design is the connection between what a designer does and the decisions the designer makes. Existing design process models, however, do not explicitly link back to models for visualization design decisions. We bridge this gap by introducing the design activity framework, a process model that explicitly connects to the nested model, a well-known visualization design decision model. The framework includes four overlapping activities that characterize the design process, with each activity explicating outcomes related to the nested model. Additionally, we describe and characterize a list of exemplar methods and how they overlap among these activities. The design activity framework is the result of reflective discussions from a collaboration on a visualization redesign project, the details of which we describe to ground the framework in a real-world design process. Lastly, from this redesign project we provide several research outcomes in the domain of cybersecurity, including an extended data abstraction and rich opportunities for future visualization research.",Sean McKenna;Dominika Mazur;James Agutter;Miriah D. Meyer,Sean McKenna;Dominika Mazur;James Agutter;Miriah Meyer,"School of Computing, Salt Lake City, UT;Department of Psychology, University of Utah, Salt Lake City, UT;College of Architecture and Planning, University of Utah, Salt Lake City, UT

In [66]:
id_title_abstract = "<b>Paper id</b>: " + dataset["id"].astype(str) + " ("  + dataset["Conference"] + ") " " - "\
"<a href=\""  + dataset.Link + "\" target=\"_blank\">Link</a>" + \
"; <br><b>Authors: </b>: " + dataset["AuthorNames"] + \
"; <br><b>Title</b>: " + dataset["Title"] + \
"; <br><b>Abstract</b>: " + dataset["Abstract"]

map_id_to_label={k: v for  k,v in enumerate(id_title_abstract)}
map_id_to_label[0]

'<b>Paper id</b>: 0 (InfoVis)  - <a href="http://dx.doi.org/10.1109/INFVIS.1995.528680" target="_blank">Link</a>; <br><b>Authors: </b>: B. Spence;L. Tweedie;H. Dawkes;Hua Su; <br><b>Title</b>: Visualisation for functional design; <br><b>Abstract</b>: We present two novel visualisation tools: the Influence Explorer and the Prosection Matrix. These were specifically created to support engineering artifact design and similar tasks in which a set of parameter values must be chosen to lead to acceptable artifact performance. These tools combine two concepts. One is the interactive and virtually immediate responsive display of data in a manner conducive to the acquisition of insight. The other, involving the precalculation of samples of artifact performance, facilitates smooth exploration and optimisation leading to a design decision. The anticipated benefits of these visualisation tools are illustrated by an example taken from electronic circuit design, in which full account must be taken o

In [67]:
df = pd.read_csv(similarity_path)
df

,id,0,1,2,3,4,5,6,7,8,...,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,1741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1742,1742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1743,1743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1744,1744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
arr = df.values
index_names = df.index
col_names = df.columns

#  Get indices where such threshold is crossed; avoid diagonal elems
R,C = np.where(np.triu(arr,1)>0.0)

# Arrange those in columns and put out as a dataframe
out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
df_out = pd.DataFrame(out_arr,columns=['from','to','value'])
df_out["from"] = df_out["from"].astype(np.int16)
df_out["to"] = df_out["to"].astype(np.int16)
df_out

,from,to,value
0,0,238,1.0
1,0,244,1.0
2,0,456,1.0
3,0,512,1.0
4,0,550,1.0
...,...,...,...
60213,1611,1612,2.0
60214,1611,1614,1.0
60215,1611,1615,2.0
60216,1614,1615,1.0


## Plot using Pyvis

In [69]:
color_dict = {
    "InfoVis": "blue",
    "VAST": "orange",
    "SciVis": "red"
}

def get_node_color(node, df, color_dict):
    conference = df[df.id == node].Conference.values[0]
    return color_dict[conference]

In [70]:
thres = 4
df_out_thres = df_out[df_out["value"] >= thres]
plotting_all_nodes = False

file_name= output_path
# g = net.Network(height='95%', width='99%',heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')

g = net.Network(height="700px", width="100%",\
                heading='Paper Similarity Using #BibliographicCouplings and #Co-citations')
g.set_template("/Users/chaupham/opt/anaconda3/envs/py38_env/lib/python3.8/site-packages/pyvis/templates/template_2.html")
g.set_edge_smooth('discrete')
# g.set_options('{"nodes": {"borderWidthSelected": "10" } }')

if plotting_all_nodes:
    node_list = list(df.index.astype(np.int16))
else:
    node_list = list(set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) )

node_labels = [ map_id_to_label.get(node) for node in node_list]
node_colors = [ get_node_color(node, dataset, color_dict) for node in node_list]
    
# g.add_nodes(node_list, title=node_labels, label=node_list, color=node_colors)

g.add_nodes(node_list, title=node_labels, color=node_colors)


edge_list = zip(df_out_thres["from"], df_out_thres["to"], df_out_thres['value'])
g.add_edges(edge_list)    
# g.show_buttons(filter_=["physics"])
# g.show_buttons()




g.set_options("""
var options =   {
        "physics": { "stabilization": false ,
            "barnesHut" : {"centralGravity":0.1,     "gravitationalConstant":-5000}
                    },
 "edges": { "smooth": { "type": "continuous" } } 
 
}
 
  """)

g.show(file_name)

#  source code: https://github.com/WestHealth/pyvis/blob/523d541d04df6779dd21e5942af60b79369e8f13/pyvis/network.py#L218

In [71]:

g.show(file_name)

In [72]:
!open $file_name

In [73]:
g.num_nodes()

628

In [74]:
import pyvis


In [75]:
pyvis

<module 'pyvis' from '/Users/chaupham/opt/anaconda3/envs/py38_env/lib/python3.8/site-packages/pyvis/__init__.py'>

In [77]:
import dill

#### Write
with open('/Users/chaupham/minhchau/MY_WORK/C/citation_networks/models/network_plot/citation_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)
    
#### Write
with open('/Users/chaupham/Downloads/streamlit_network-chau/models/citation_similarity.pkl', 'wb') as f:     
    dill.dump(g, f)

In [ ]:
file_name

In [ ]:
!open .

## Plot using Jaal

More elegant and faster, but still in its infancy though. There's a lack of some crucial features such as "edge thickness"

In [ ]:
dataset

In [ ]:
import pandas as pd
from jaal import Jaal
from jaal.datasets import load_got


thres = 6
plotting_all_nodes = False


    
    

df_out_thres = df_out[df_out["value"] >= thres]
edges = df_out_thres

(dataset)
# edges.loc[:, 'label'] = edges.loc[:, 'value'].astype(int).astype(str)

edges["from"] = edges["from"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0]))
edges["to"] = edges["to"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 


set_thres_nodes = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) )
nodes = pd.DataFrame({"id": df.index})
nodes["having_edge"] = nodes["id"].map(lambda x: x in set_thres_nodes).astype(str)
nodes["conference"] = nodes["id"].map(lambda node_id: dataset[dataset["id"] == node_id ]["Conference"].values[0])
nodes["Title"] = nodes["id"].map(lambda node_id: " - ".join(dataset[dataset["id"] == node_id ][["Title", "AuthorNames"]].values[0])) 
nodes["id"] = nodes["Title"]
if plotting_all_nodes:
    pass
else:
    print(nodes.shape)
    node_list = set.union( set(df_out_thres["from"]), set(df_out_thres["to"]) ) 
    nodes = nodes[nodes["id"].isin(node_list)]
    print(nodes.shape)

nodes.to_csv("../data/interim/jaal_data/nodes_2.csv", index=False)
edges.to_csv("../data/interim/jaal_data/edges_2.csv", index=False)

port=8050

while True:
    try:
        Jaal(edges, nodes).plot(port=port)
    except:
        port+=1


In [ ]:
edges["value"] = edges["value"].astype(int)

In [ ]:
edges.nlargest(10,"value")

In [ ]:
node_list

In [ ]:
nodes[nodes.id==1606].Title.values[0]

In [ ]:
edges[(edges["to"] == 755) & (edges["from"]==456)]

In [ ]:
pip install networkx

In [ ]:
import networkx as nx
from pyvis.network import Network


nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)
nx_graph.add_edge(20, 21, weight=5)
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)

nt = Network(notebook=True, height="750px", width="100%")

nt.from_nx(nx_graph)
nt.show("nx.html")